## Training and deploying Machine Learning models 

In [21]:
import pandas as pd

#Loading the wine quality dataset 
df = pd.read_csv("winequality-red.csv",delimiter=";")

# Removing the label column ---> wine quality
label =df['quality']

#The remaining columns are features
features =df.drop('quality',axis=1)

#label.unique()
label.value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

In [22]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


### Build the model - Linear Regression

In [26]:
import numpy as np
from sklearn import linear_model

#defining our Linear Regression estimator and trainig it with wine data

regr = linear_model.LinearRegression()
regr.fit(features,label)

#Let us use our trained model to detect a Fake wine
preds=regr.predict([[7.4,0.66,0,1.8,0.075,13,40,0.9978,3.51,0.56,9.4]]).tolist()
print(preds)

[5.0655703490749175]


### Importing and exporting our Python Model

In [36]:
# We use the pickle library to serialize the models into files.
# I can also load the model back into the code
import pickle

#Serializing our model to a file called model.pkl
#serializing our model to a file called model.pkl
pickle.dump(regr, open("model.pkl","wb"))

#loading a model from a file called model.pkl
model = pickle.load(open("model.pkl","rb"))

#### The file is read in Byte format indicated by rb and wb

### We now create a web server using Flask Python framework

In [40]:
from flask import Flask
from flask import request

# To initialize our server
app = Flask(__name__)

# Defining a /hello route for only post requests
@app.route('/hello', methods=['POST'])

def index():
    #grabs the data tagged as 'name'
    name = request.get_json()['name']
    
    #sending a Hello back to requester
    return "Hello" + name


### Add model to the server

In [41]:
@app.route('/predict',methods=['POST'])
def predict():
    #Take a set of wine features from the request's body
    feature_array= request.get_json()['feature_array']
    
    # Perform prediction
    prediction = model.predict([feature_array]).tolist()
    
    #preparing the response and sending it as json
    response={}
    response['predictions'] = prediction
    
    return flask.jsonify(response)
    